In [3]:
import scipy

def ensemble_approach(repeat_accuracy,*test_X,test_y,model):
    repeat_accuracy=np.array(repeat_accuracy).mean(1)
    index = np.where(repeat_accuracy == np.amax(repeat_accuracy))[0][0]
    print(index)
  
    ens_y_pred=[]
    ens_y_pred_proba=[]

    for i in range(5):
        weights_file='weights/' + model.name + str(index) + '_' + str(i) +'_best_pretrained_weights.npy'
        model.load_weights(weights_file)
        prediction=model.predict([test_X[0], test_X[1], test_X[2]])
        y_pred = (prediction > 0.5).argmax(axis=1)
        y_pred_proba=prediction[:, 1]
        ens_y_pred.append(y_pred)
        ens_y_pred_proba.append(y_pred_proba)
        
    new_ens_y_pred=np.transpose(np.array(ens_y_pred),(1,0))
    new_ens_y_pred=scipy.stats.mode(new_ens_y_pred,axis=1)[0]
    ens_y_true=test_y
    ens_matrix=metrics.confusion_matrix(ens_y_true,new_ens_y_pred)
    print(ens_matrix)
    #accuracy
    total = sum(sum(ens_matrix))
    ens_accuracy = (ens_matrix[0, 0] + ens_matrix[1, 1]) / total

    #sensitivity & specificity
    ens_specificity=ens_matrix[0, 0] / (ens_matrix[0, 0] + ens_matrix[0, 1])
    ens_sensitivity=ens_matrix[1, 1] / (ens_matrix[1, 0] + ens_matrix[1, 1])

    print("%.2f%% " % (np.mean(ens_accuracy)*100))
    print("%.2f%% " % (np.mean(ens_sensitivity)*100))
    print("%.2f%% " % (np.mean(ens_specificity)*100))
    
    roc_y_pred=np.zeros(np.array(ens_y_pred).shape)
    for i in range (roc_y_pred.shape[0]):
        for j in range (roc_y_pred.shape[1]):
            roc_y_pred[i][j]=1-abs(ens_y_pred[i][j]-new_ens_y_pred[j])
    
    roc_y_pred_proba=ens_y_pred_proba*roc_y_pred
    roc_y_pred_proba=np.sum(roc_y_pred_proba, axis=0)/np.sum(roc_y_pred,axis=0)
    
    test_y_probability=roc_y_pred_proba
    test_y_label=ens_y_true
    return test_y_probability,test_y_label

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

def plot_roc_curve(y_true,y_pred):
    y_true=np.array(y_true).flatten()
    y_pred=np.array(y_pred).flatten()
    auc = roc_auc_score(y_true,y_pred)
    print('AUC: %.3f' % auc)

    # calculate roc curve
    fpr, tpr, thresholds = roc_curve(y_true,y_pred)
    # plot no skill
    pyplot.plot([0, 1], [0, 1], linestyle='--')
    # plot the roc curve for the model
    pyplot.plot(fpr, tpr, marker='.')
    # show the plot
    pyplot.show()

In [ ]:
plot_roc_curve(ens_y_true,roc_y_pred_proba)